In [2]:
pip install -U sentence-transformers

  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/08/aa/033a0a1bca4a3b8f81f97709024e053da37cf5c970f3dc5d76233c920b99/torchvision-0.16.0-cp311-cp311-manylinux1_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.1 MB/s eta 0:00:00m eta 0:00:010:01:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 5.1 MB/s eta 0:00:00m eta 0:00:010:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=96f4ab7c7e3ee624aa6bdaa05dae82d14425330fec6bd4ad05df22eeb3855695
  Stored in directory: /home/joby/.cache/pip/wheels/ff/27/bf/ffba8b318b02d7f691a57084ee154e26ed24d012b0c7805881
Successfully built sentence-transformers


Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch

In [11]:
from transformers import AutoModel, AutoTokenizer

model_name = "bert-base-uncased"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [12]:
with open('DP_Sample.json', 'r') as file:
    data = json.load(file)

tokenized_data = [tokenizer(item, truncation=True, padding=True, return_tensors="pt") for item in data]


In [15]:
embeddings = []

for item in tokenized_data:
    with torch.no_grad():
        outputs = model(**item)
        embeddings.append(outputs.last_hidden_state)


In [16]:
embeddings

[tensor([[[-0.4156,  0.1556,  0.0974,  ..., -0.3049, -0.0580,  0.2635],
          [ 0.0055,  0.4952,  0.5756,  ..., -0.8292, -0.4116, -0.4782],
          [ 0.8726,  0.0712, -0.2783,  ..., -0.1688, -0.8929, -0.3399]]]),
 tensor([[[-0.5760,  0.0350, -0.2270,  ..., -0.1851,  0.0940,  0.3281],
          [-0.4651, -0.4263, -0.2407,  ...,  0.1948,  0.1887,  0.1204],
          [ 0.8748,  0.1016, -0.4328,  ...,  0.1037, -0.5909, -0.2674]]]),
 tensor([[[-0.5507,  0.0664,  0.2064,  ...,  0.0382,  0.1508,  0.3129],
          [-0.5916, -0.6735,  0.1297,  ...,  0.2691,  0.5061,  0.3456],
          [ 0.9693,  0.0079, -0.2108,  ...,  0.1358, -1.0094, -0.1803]]]),
 tensor([[[-0.3529, -0.2390,  0.1322,  ...,  0.0110,  0.1277,  0.2494],
          [-0.3776, -1.1474, -0.0256,  ..., -0.0542, -0.0551,  0.3992],
          [ 0.7580,  0.0903, -0.2518,  ..., -0.0291, -0.9104, -0.3047]]]),
 tensor([[[-0.3530, -0.1299,  0.1074,  ..., -0.3348,  0.1649,  0.5878],
          [-0.0378, -0.6815, -0.1101,  ..., -0.4189,

In [17]:
item_index = 0  
embeddings_for_item = embeddings[item_index]

print("Shape of embeddings:", embeddings_for_item.shape)

print("Embeddings:", embeddings_for_item)


Shape of embeddings: torch.Size([1, 3, 768])
Embeddings: tensor([[[-0.4156,  0.1556,  0.0974,  ..., -0.3049, -0.0580,  0.2635],
         [ 0.0055,  0.4952,  0.5756,  ..., -0.8292, -0.4116, -0.4782],
         [ 0.8726,  0.0712, -0.2783,  ..., -0.1688, -0.8929, -0.3399]]])


In [25]:
with open('DP_Sample.json', 'r') as file:
    data = json.load(file)

text_fields = [
    "title",
    "description",
    "tags",  
    "document",
]

data_sources = data.get("data_sources", [])

for item in data_sources:
    text_fields.extend([item["title"], item["description"]])

embeddings_dict = {}

for field in text_fields:
    text = data.get(field, "")  

    input_ids = tokenizer.encode(text, add_special_tokens=True, return_tensors="pt")

    with torch.no_grad():
        outputs = model(input_ids)
        embeddings = outputs.last_hidden_state[0]  

    embeddings_list = embeddings.tolist()

    embeddings_dict[field] = embeddings_list

with open("field_embeddings.json", "w") as file:
    json.dump(embeddings_dict, file)



In [38]:
def text2vec(text):
    
    
    return vec

In [47]:
import torch
import json

def calculate_field_medians(data, text_fields, tokenizer, model):
    embeddings_dict = {}

    for field in text_fields:
        text = data.get(field, "")

        input_ids = tokenizer.encode(text, add_special_tokens=True, return_tensors="pt")

        with torch.no_grad():
            outputs = model(input_ids)
            embeddings = outputs.last_hidden_state[0]

        median_embedding = torch.median(embeddings, dim=0).values.unsqueeze(0)
        embeddings_list = median_embedding.tolist()

        embeddings_dict[field] = embeddings_list

    return embeddings_dict


with open('DP_Sample.json', 'r') as file:
    data = json.load(file)

text_fields = [
    "title",
    "description",
    "tags",  
    "document",
]

data_sources = data.get("data_sources", [])

for item in data_sources:
    text_fields.extend([item["title"], item["description"]])

field_embeddings = calculate_field_medians(data, text_fields, tokenizer, model)

with open("field_medians.json", "w") as file:
    json.dump(field_embeddings, file)


In [71]:
import json
from sentence_transformers import SentenceTransformer


In [72]:
with open('DP_Sample.json', 'r') as file:
    data = json.load(file)


In [73]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [74]:
from ast import literal_eval

In [75]:
type(json_text)

str

In [76]:
literal_eval(json_text)

{'title': 'Customer_360',
 'owner': 'Ishmin Singh',
 'tier': 'Gold',
 'version': 'v1.1',
 'version_date': 'July 17, 2023 13:00',
 'description': 'Customer data from various sources, enabling a holistic view of customers. ',
 'tags': ['marketing', 'customer.360'],
 'Health': {'quality_score': '70%', 'operational_health': '70%'},
 'governance': {'access_policies': 3, 'data_policies': 3},
 'stacks': ['flare', 'benthos', 'alpha'],
 'document': 'What is it?\nThe Customer 360 is a tool designed to create a comprehensive view of customers by combining data from multiple sources. It includes information centered around a customer and further associated details of stores, products, and transaction details. \n\nWhat does it solve? \nCustomer 360 addresses the challenge of aggregating and analyzing customer-related data from disparate sources. By consolidating data into a unified format, it enables users to gain valuable insights into customer behavior, purchase patterns, and preferences. This co

In [77]:
specific_keys = ['title', 'description', 'tags', 'document']

In [78]:
encoded_data = {}

In [79]:
for key in specific_keys:
    if key in data and isinstance(data[key], str):
        encoded_data[key] = model.encode(data[key])

In [80]:
encoded_data_sources = []

In [81]:
for source in data.get('data_sources', []):
    if isinstance(source, dict):
        encoded_source = {}
        for source_key in ['title', 'description']:
            if source_key in source and isinstance(source[source_key], str):
                encoded_source[source_key] = model.encode(source[source_key])
        if encoded_source:
            encoded_data_sources.append(encoded_source)

In [83]:
print(encoded_data_sources)

[{'title': array([-6.59204200e-02,  3.12766014e-03,  9.41269100e-03, -2.46390775e-02,
       -8.75265822e-02, -1.19773811e-02,  1.58399791e-01, -1.16684865e-02,
        3.20763215e-02, -6.68867081e-02,  8.34070444e-02,  1.50712281e-02,
        1.83223113e-02, -1.29731214e-02,  3.34732160e-02, -3.98448110e-02,
        4.10473719e-02, -2.30349856e-03, -4.42626141e-02, -2.28488352e-02,
       -1.46660790e-01, -3.37006152e-02, -7.87534863e-02,  1.37549303e-02,
       -4.91207875e-02,  3.81494723e-02, -7.67282560e-04,  5.68842143e-02,
       -2.96176951e-02, -7.12712482e-02,  4.02667699e-03,  2.88355462e-02,
        1.32891774e-01, -5.11289363e-05,  2.34600920e-02,  2.75689233e-02,
       -4.32844758e-02, -1.53212305e-02,  2.99984179e-02,  1.08069312e-02,
       -1.18952701e-02, -6.20268732e-02, -8.97834376e-02,  5.47086401e-03,
        7.90256932e-02, -3.02648861e-02,  5.25777005e-02,  9.01008844e-02,
        6.00898191e-02,  6.39852732e-02, -4.99101728e-02, -2.28416361e-02,
       -3.3826

In [82]:
import csv

with open('embeds.csv', 'w', newline='') as csv_file:
    fieldnames = ['Key', 'Embedding']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    for key, embeddings in encoded_data.items():
        writer.writerow({'Key': key, 'Embedding': embeddings.tolist()})

    for i, source in enumerate(encoded_data_sources):
        for key, embeddings in source.items():
            writer.writerow({'Key': f'Data Source {i + 1} - {key}', 'Embedding': embeddings.tolist()})

In [84]:
import json

json_file_path = 'embeddings.json'

data_to_save = {}

for key, embeddings in encoded_data.items():
    data_to_save[key] = embeddings.tolist()

data_to_save['Data Sources'] = []
for i, source in enumerate(encoded_data_sources):
    source_data = {}
    for source_key, embeddings in source.items():
        source_data[source_key] = embeddings.tolist()
    data_to_save['Data Sources'].append(source_data)

with open(json_file_path, 'w') as json_file:
    json.dump(data_to_save, json_file, indent=4)


In [3]:
import json
with open('embeddings.json', 'r') as file:
    embedding = json.load(file)


In [5]:
embedding

{'title': [-0.0750977098941803,
  -0.032000958919525146,
  -0.03437187150120735,
  -0.054321449249982834,
  0.009169877506792545,
  -0.04182756319642067,
  0.10451316833496094,
  0.009180744178593159,
  0.017489438876509666,
  -0.06019215285778046,
  0.07124654948711395,
  -0.020160837098956108,
  0.034009113907814026,
  -0.02023700438439846,
  0.03482156991958618,
  -0.02517235279083252,
  -0.0056594256311655045,
  0.02753726951777935,
  -0.10023246705532074,
  0.006730086635798216,
  -0.1408040076494217,
  0.0020834109745919704,
  -0.06547527760267258,
  0.05336359515786171,
  -0.05474458634853363,
  0.041959114372730255,
  0.007732254918664694,
  0.05553324520587921,
  -0.052701301872730255,
  -0.044167377054691315,
  0.005193445831537247,
  0.03232532739639282,
  0.11941839009523392,
  0.006203310564160347,
  0.04273542761802673,
  -0.027671033516526222,
  -0.06385061889886856,
  -0.026103731244802475,
  -0.08253176510334015,
  -0.010413718409836292,
  -0.059735074639320374,
  -0.0

In [21]:
import json

with open('DP_Sample.json', 'r') as file1:
    data1 = json.load(file1)

with open('embeddings.json', 'r') as file2:
    data2 = json.load(file2)

title1 = data1["title"]
description1 = data1["description"]

title_vectors = data2["title"]
description_vectors = data2["description"]


combined_dataprod = {
    "title": title1,
    "description": description1,
    "title_vec": title_vectors,
    "description_vec": description_vectors,
}

In [40]:
combined_dataprod

{'title': 'Customer_360',
 'description': 'Customer data from various sources, enabling a holistic view of customers. ',
 'title_vec': [-0.0750977098941803,
  -0.032000958919525146,
  -0.03437187150120735,
  -0.054321449249982834,
  0.009169877506792545,
  -0.04182756319642067,
  0.10451316833496094,
  0.009180744178593159,
  0.017489438876509666,
  -0.06019215285778046,
  0.07124654948711395,
  -0.020160837098956108,
  0.034009113907814026,
  -0.02023700438439846,
  0.03482156991958618,
  -0.02517235279083252,
  -0.0056594256311655045,
  0.02753726951777935,
  -0.10023246705532074,
  0.006730086635798216,
  -0.1408040076494217,
  0.0020834109745919704,
  -0.06547527760267258,
  0.05336359515786171,
  -0.05474458634853363,
  0.041959114372730255,
  0.007732254918664694,
  0.05553324520587921,
  -0.052701301872730255,
  -0.044167377054691315,
  0.005193445831537247,
  0.03232532739639282,
  0.11941839009523392,
  0.006203310564160347,
  0.04273542761802673,
  -0.027671033516526222,
  -0

In [53]:
data_sources = data1["data_sources"]

data_vectors = data2["Data Sources"]

combined_dataset = {
    "data_sources": data_sources,
    "data_vectors": data_vectors
}

In [54]:
combined_dataset

{'data_sources': [{'title': 'customers',
   'description': 'Contains customer information, including customer_index, name, contact details, and other relevant attributes. '},
  {'title': 'stores',
   'description': 'Provides data about various stores, including store_index, location, and store attributes.'},
  {'title': 'products',
   'description': 'Contains details of different products, including product_index, name, description, and product attributes. '},
  {'title': 'transactions',
   'description': 'Holds transaction data, including transaction_index, transaction_date, transaction_line_item, and transaction details.'}],
 'data_vectors': [{'title': [-0.06592041999101639,
    0.003127660136669874,
    0.009412690997123718,
    -0.024639077484607697,
    -0.0875265821814537,
    -0.011977381072938442,
    0.1583997905254364,
    -0.011668486520648003,
    0.03207632154226303,
    -0.06688670814037323,
    0.08340704441070557,
    0.01507122814655304,
    0.018322311341762543,
    -

In [35]:
with open('combined_dataprod.json', 'w') as outfile:
    json.dump(combined_dataprod, outfile, indent=4)

In [36]:
with open('combined_dataset.json', 'w') as outfile:
    json.dump(combined_dataset, outfile, indent=4)